# NeurIPS 2023问答agent（未完成）
+ 主要特点
    + 跨领域、文档的全面分析
    + 相关内容的理解和定位
+ 例子：
    > “基于GNN的多跳推理在这次会议上有新的进展吗？”
    > “因果推理用于改善决策的问题有什么新思路吗？”

说明：    
+ 需要安装redis
+ 用了clash进行本地代理

# 数据采集

In [1]:
# NeurIPS 2023的论文标题列表
import re, requests, aiohttp, asyncio, os, pickle, json, time, redis, re, random, hashlib
from fn import F
from returns.future import Future
from tqdm.asyncio import tqdm
from bs4 import BeautifulSoup
os.environ['http_proxy'] = "http://127.0.0.1:7890"
os.environ['https_proxy'] = "http://127.0.0.1:7890"

neurips_url = 'https://neurips.cc/virtual/2023/papers.html?filter=titles'
titles = re.findall(r'<a href="[^"]*">(.+?)</a>', requests.get(neurips_url).text)

## 并发API获取和存储

In [6]:
import httpx

redis_client = redis.Redis(db=15)


# 一个通用异步网络请求框架
# fetch逻辑是把data参数传给invoker然后获取response。invoker可以是任意的请求调用函数


def build_fetcher(invoker):
    ...
    async def fetch(**args):
        # async with semaphore:
        assert 'semaphore' in args.keys()
        async with args['semaphore']:
            try:
                async with await invoker(**args) as response: #coroutine
                    if response.status == 200:
                        res = await response.text()
                        assert 'data' in args.keys()
                        return {'input':args['data'], 'response':res}
                    else:
                        error_message = await response.text()
                        print(f"Request failed: {response.status}. Error message: {error_message}")
            except Exception as e:
                print(f"error: {e}")
    fetch.url = invoker.url
    return fetch

# 并发调用fetch&然后缓存到redis的一种实现方式。基于输入的data用key模板函数分配唯一的key。
# data → fetch → (key, result) → redis
# 会给invoker的每个协程分配session和同步信号量

def fetch_spawn_concurrent(fetcher, key_template, workers=10):
    async def _fetch_spawn_concurrent(inputs):
        
        key_set = set(map(key_template, inputs))
        print('total:', len(key_set))
        print('to fetch: ', len(set(filter(lambda x:not redis_client.exists(x), key_set))))
        
        semaphore = asyncio.Semaphore(workers)
        async with aiohttp.ClientSession() as session:
        # async with httpx.AsyncClient(timeout=5,
        #                              proxies={"http://": "http://127.0.0.1:7890", 
        #                                       "https://": "http://127.0.0.1:7890"}
        #                              ) as invoker_client:
            tasks = [fetcher(data=input, 
                             semaphore=semaphore, 
                             session=session) for input in inputs 
                     if not redis_client.exists(key_template(input))]
            for f in tqdm.as_completed(tasks, total=len(tasks), desc=f'[Fetching] {fetcher.url}'):
                result = await f
                match result:
                    case {'input':input, 'response':_}:
                        redis_client.set(key_template(input), json.dumps(result))
    return _fetch_spawn_concurrent

# 用于元数据的redis key模板函数
# data → key
def key_template_metadata(source, conf):
    def _key_template(data):
        _data = re.sub('[^\w\-_\s]', '', re.sub(':', '-', data))
        return f"{source}:{conf}:{_data}"
    return _key_template

# 定义了采集不同数据的redis key模板函数
key_template_neurips_abs = key_template_metadata('neurips:abs', 'NeurIPS-2023')
key_template_arxiv = key_template_metadata('arxiv', 'NeurIPS-2023')
key_template_openreview = key_template_metadata('openreview', 'NeurIPS-2023')


## 获取官方元数据

In [11]:
os.environ['http_proxy'] = "http://127.0.0.1:7890"
os.environ['https_proxy'] = "http://127.0.0.1:7890"

# 流程是根据title获取abstract的相对路径，然后拼成完整路径用fetch_spawn_concurrent并发调用

abs_urls, titles_ = zip(*re.findall(r'<a title="paper title" href="([^"]*)">(.+?)</a>', 
                                    requests.get('https://papers.nips.cc/paper_files/paper/2023').text))

abs_urls = list(map(lambda x:x.replace('_files/paper', ''), abs_urls))
# title到abs相对路径的映射
title2abs = dict(zip(titles_, abs_urls))

NeurIPS_ABS_API_URL = 'https://papers.nips.cc/paper_files'

# async def invoker_neurips_abs(client, data, **kargs):
#     return client.stream('GET', url=NeurIPS_ABS_API_URL+title2abs[data])

#只需要定义调用请求的invoker和存在redis的key模板函数，就可以用并发函数执行了

async def invoker_neurips_abs(session, data, **kargs):
    return session.get(url=NeurIPS_ABS_API_URL+title2abs[data],
                       proxy="http://127.0.0.1:7890")

invoker_neurips_abs.url = NeurIPS_ABS_API_URL
fetcher_neurips_abs = build_fetcher(invoker_neurips_abs)

await fetch_spawn_concurrent(fetcher_neurips_abs, key_template_neurips_abs, workers=10)(titles_)

# 单个用例的情况：
# semaphore = asyncio.Semaphore(1)
# async with aiohttp.ClientSession() as session:
#     z = await fetcher_neurips_abs(titles[0], semaphore, session)


total: 3540
to fetch:  0


[Fetching] https://papers.nips.cc/paper_files: 0it [00:00, ?it/s]


In [ ]:
# 随机删掉十条记录然后测试上面的并发调用
[redis_client.delete(x) for x in random.sample(redis_client.keys('neurips*'), 10)]
!rma -s localhost -p 6379 -d 15

## arxiv、openreview元数据

In [7]:
# 原理和官方元数据的调用基本相同。请求的参数会有区别

ARXIV_API_URL = "http://export.arxiv.org/api/query"
OPENREVIEW_API_URL = "https://api2.openreview.net/notes/search"


async def invoker_arxiv(session, data, **kargs):
    return session.get(url=ARXIV_API_URL, 
                       proxy="http://127.0.0.1:7890", 
                       params={'search_query':f'ti:{data}', 'start':0, 'max_results':1})
invoker_arxiv.url = ARXIV_API_URL


async def invoker_openreview(session, data, **kargs):
    return session.get(url=invoker_openreview.url, 
                       proxy="http://127.0.0.1:7890", 
                       params={"term": data, "offset": 0, "limit": 1})
invoker_openreview.url = OPENREVIEW_API_URL

fetcher_arxiv = build_fetcher(invoker_arxiv)
fetcher_openreview = build_fetcher(invoker_openreview)

await fetch_spawn_concurrent(fetcher_arxiv, key_template_arxiv, workers=10)(titles)
# await fetch_spawn_concurrent(fetcher_openreview, key_template_openreview, workers=1)(titles)

total: 3584
to fetch:  0


[Fetching] http://export.arxiv.org/api/query: 0it [00:00, ?it/s]


In [ ]:
#同样是随机删掉十条记录用于测试
[redis_client.delete(x) for x in random.sample(redis_client.keys('arxiv*'), 10)]
# [redis_client.delete(x) for x in redis_client.keys('research_topic*')]
!rma -s localhost -p 6379 -d 15

## 数据清洗、处理

In [8]:
from returns.context import Reader
from returns.unsafe import unsafe_perform_io
from returns.maybe import Maybe, Nothing

# 定义了一个Maybe的monad chain用于元数据后处理。输入：title，输出：采集的内容。
# Maybe chain用于处理每一步可能出现的空值异常。暂时直接过滤了没有填充缺失值

metadata_chain = lambda key_template: lambda x: (((Maybe.from_optional(x)
                                         .bind_optional(key_template))
                                         .bind_optional(lambda x:redis_client.get(x)))
                                         # .or_else_call(...)
                                         # .alt(...)
                                         .bind_optional(lambda x:x.decode('utf-8'))
                                         .bind_optional(lambda x:json.loads(x).get('response')))

post_process = lambda metadata: {x[0]:x[1].unwrap() for x in filter(lambda x:x[1]!=Nothing, metadata)}

def mapper():
    ...

In [9]:
import xmltodict
from difflib import SequenceMatcher
from returns.pipeline import flow

# arxiv_chain = lambda x:((metadata_chain(x, key_template_arxiv)
#                         .bind_optional(xmltodict.parse))
#                         .bind_optional(lambda x:x.get('feed'))
#                         .bind_optional(lambda x:x.get('entry'))
#                         # .or_else_call({'entry': x}))
#                         .bind_optional(lambda x:{'entry':x, 'arxiv_title':x.get('title')}))
# 
# 
# metadata_arxiv = list(map(lambda x: (x, arxiv_chain(x)), titles))

#用于解析arxiv请求结果的chain。输入：采集的响应数据，输出：元数据json。

arxiv_chain = lambda x:((Maybe.from_optional(x)
                        .bind_optional(xmltodict.parse))
                        .bind_optional(lambda x:x.get('feed'))
                        .bind_optional(lambda x:x.get('entry'))
                        # .or_else_call({'entry': x}))
                        .bind_optional(lambda x:{'entry':x, 'arxiv_title':x.get('title')}))

#将metadata_chain和arxiv_chain拼起来得到title→元数据的chain

metadata_arxiv = list(map(lambda x: (x, flow(x, 
                                             metadata_chain(key_template_arxiv), 
                                             lambda x:x.unwrap(), 
                                             arxiv_chain)), titles))

metadata_arxiv = post_process(metadata_arxiv)

# 过滤query和检索结果的title不一致的情况
metadta_arxiv_filter = lambda x:SequenceMatcher(None, x[0], x[1].get('arxiv_title')).ratio() > 0.9
metadata_arxiv = dict(filter(metadta_arxiv_filter, metadata_arxiv.items()))


In [12]:
# 官方元数据解析的chain。title→元数据
metadata_neurips_abs = list(map(lambda x:
                                (x,metadata_chain(key_template_neurips_abs)(x)
                                 .bind_optional(lambda x: re.findall(r'<h4>Abstract</h4>\s*<p>(?:<p>)?([\s\S]*?)<\/p>',x))
                                 ),
                                titles))

metadata_neurips_abs = post_process(metadata_neurips_abs)

In [358]:
import anyio
import asyncio
from functools import partial

# 这里说明了独立定义这些chain的好处。可以进行各种业务逻辑的形式化组合。
# 比如说用future(一种monad)在arxiv检索结果和query不一致的时候用其他方式获取数据

async def fetch_chain(title):
    semaphore = asyncio.Semaphore(10)
    async with aiohttp.ClientSession() as session:
        
        fetcher_arxiv_ = partial(fetcher_arxiv, semaphore=semaphore, session=session)
        result = await (Future.from_value(title)
                        .bind_awaitable(fetcher_arxiv_)
                        .map(lambda x:x.get('response'))
                        .map(arxiv_chain)
                        .map(...)
                        .awaitable())
        return result

z = await asyncio.get_event_loop().run_in_executor(None, anyio.run, fetch_chain)
z1 = unsafe_perform_io(z).unwrap()


# 基于Claude的问答数据集合成
+ 用Claude总结所有的研究主题。输入：标题列表，输出：一组主题名称
+ 为每个研究主题构造一些问题
+ 用Retriever获取问题的相关论文
+ 用Claude评估问题和论文的相关性
+ 相关性用于训练RAG模型
# RAG pipeline
+ 基于Bi-encoder的初排（语义相似度）
+ 基于Cross-encoder的重排（阅读理解）

## 总结NeurIPS 2023的主题

In [299]:
# titles_id = list(enumerate(metadata_neurips_abs.keys()))
# id2title = dict(titles_id)
# ids = id2title.keys()
# titles = list(id2title.values())
# abs_text = list(map(lambda x:metadata_neurips_abs[x][0], titles))


# 基于标题列表生成主题名称

In [54]:
import base64

# 把完整标题列表分块调用Claude生成主题，最后再合并结果

chunk_sz = 400

prompt_topic_gen = """
Task: Analyze and categorize paper titles from NeurIPS 2023

Dataset: 3500 paper titles, to be processed in batches of {{chunk_sz}}

For each batch, perform the following:

1. Identify coarse-grained research topics. The research topics should not be defined too broadly (such as machine learning), but need to have a certain degree of distinction (e.g., In-context learning, Graph Contrastive Learning, etc.)
2. Associate each paper with the identified areas and topics

Input format: List of paper titles with corresponding IDs

Output format: JSON structure as follows:

  "research_topics": {
    "topic1": ["paper_id1", "paper_id2", ...],
    "topic2": [...],
    ...
  }
}

Batch 1 of paper titles:

{{titles}}

Provide the analysis for this batch. Subsequent batches will build upon the areas and topics identified in previous iterations.
""".replace('{{chunk_sz}}', str(chunk_sz))

key_list = list(enumerate(metadata_neurips_abs.keys()))

# 为每个块生成输入的context
contexts_topic = list(map(lambda x: ((Maybe.from_optional(x)
                               .bind_optional(lambda x:json.dumps(key_list[x*chunk_sz:(x+1)*chunk_sz])))
                              .bind_optional(lambda x: prompt_topic_gen.replace('{{titles}}', x))
                              .unwrap()),
                   range(len(metadata_neurips_abs)//chunk_sz+1)))

# claude api的并发调用

token = os.environ['CLAUDE_API_token']
model = "claude-3-5-sonnet-20240620"

# CLAUDE_API_URL = 'https://api.anthropic.com/v1/messages'
CLAUDE_API_URL = 'https://api.gptapi.us/v1/chat/completions' # 某个更稳定的国内CLAUDE中转


async def invoker_claude(session, data, **kargs):
    return session.post(url=CLAUDE_API_URL,
                        headers={'Content-Type': 'application/json',
                                 'Authorization': f'Bearer {token}'},
                        json={"model": model,
                                 "max_tokens": 4096,
                                 "messages": [{"role": "user", "content": data}]})

invoker_claude.url = CLAUDE_API_URL
fetcher_claude = build_fetcher(invoker_claude)
redis_client.delete("research_topic:claude")

# 对于每个输入context用base64编码作为redis的key
key_template_topic = lambda prompt: f"research_topic:claude:{base64.b64encode(prompt.encode()).decode()}"

# await fetch_spawn_concurrent(fetcher_claude, key_template_topic, workers=10)(contexts_topic)

In [ ]:
[redis_client.delete(x) for x in redis_client.keys('research_topic*')]
# [redis_client.delete(x) for x in redis_client.keys('question*')]
!rma -s localhost -p 6379 -d 15

In [26]:
# Claude结果后处理
claude_chain = lambda x, key_template: (Maybe.from_optional(x)
                                        .bind_optional(key_template)
                                        .bind_optional(lambda x:redis_client.get(x))
                                        .bind_optional(lambda x:x.decode('utf-8'))
                                        .bind_optional(lambda x:json.loads(x)['response'])
                                        .bind_optional(json.loads)
                                        .bind_optional(lambda x:x['choices'][0]['message']['content'])
                                        )

# 主题合并

In [51]:
from itertools import chain, groupby

topic2papers = list(map(lambda x:
                       claude_chain(x, key_template_topic)
                       .bind_optional(lambda x: re.findall(r'{[^{]*?}', x))
                       .bind_optional(lambda x:x[0])
                       .bind_optional(lambda x: json.loads(x))
                       .unwrap(),
                       contexts_topic))

topic2papers_flatten = list(chain(*chain(*map(lambda x:
                                            [[(a,c) for c in b] for a,b in x.items()], topic2papers))) )

topic2papers = {x:list([z[1] for z in y]) for x,y in groupby(
    sorted(topic2papers_flatten, key=lambda x:x[0]), key=lambda x:x[0]
)}
# topic2title = {x:[id2title[int(z)] for z in y] for x,y in topic2id.items()}

## 为每个主题生成问题

In [56]:
prompt_question_gen = """
You are now taking on the role of an AI researcher.
There is a LLM-based question-answering agent specifically designed for NeurIPS 2023. It can answer questions about the academic content of NeurIPS 2023 based on RAG technology.

Now, consider the following research topics and their corresponding paper titles:
{{topics}}

Your task is:
From the perspective of researchers in each of the above topics, conceive 5-7 questions related to NeurIPS 2023 based on these research topics. These questions should help you understand the trends in the mentioned research areas at this conference or be helpful for your research. The questions need to be clear, specific, practically meaningful, and in-depth. Please ensure the questions are diverse, including but not limited to:
- Analysis of research trends
- Discussion of technical details
- Comparison with previous years
- Cross-domain applications
- Current challenges
- Future research directions

Return the results in the following JSON format:
{
  "Topic1": [
    "Question1",
    "Question2",
    ...
  ],
  "Topic2": [
    "Question1",
    "Question2",
    ...
  ],
  ...
}

Examples of questions:
1. "Which GNN-related topics received significant attention at this conference?"
2. "Were there any new developments in GNN-based multi-hop reasoning presented at this conference?"
3. "Compared to last year's NeurIPS, in which application areas has GNN shown notable expansion?"
4. "What new discoveries were presented at this conference regarding the combination of GNNs and large language models?"
5. "What are the main challenges GNNs still face when dealing with dynamic graph structures?"
"""

chunk_sz_question = 10

key_list2 = list(enumerate(topic2papers.keys()))

prompts_question = list(map(lambda x: (Maybe.from_optional(x)
                                       .bind_optional(lambda x:json.dumps(key_list2[x*chunk_sz_question:(x+1)*chunk_sz_question]))
                                       .bind_optional(lambda x: prompt_question_gen.replace('{{topics}}', x))
                                       .unwrap()),
                   range(len(key_list2)//chunk_sz_question+1)))

key_template_question = lambda prompt: f"question:claude:{base64.b64encode(prompt.encode()).decode()}"

await fetch_spawn_concurrent(fetcher_claude, key_template_question, workers=10)(prompts_question)

total: 10
to fetch:  10


[Fetching] https://api.gptapi.us/v1/chat/completions: 100%|██████████| 10/10 [00:22<00:00,  2.28s/it]


In [62]:
topic2question = list(map(lambda x:
                       claude_chain(x, key_template_question)
                       .bind_optional(lambda x: re.findall(r'{[^{]*?}', x))
                       .bind_optional(lambda x: x[0])
                       .bind_optional(lambda x: json.loads(x))
                       .unwrap(),
                       prompts_question))

questions = list(chain(*chain(*[list(x.values()) for x in topic2question])))

# 论文摘要的embedding

In [278]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm as sync_tqdm
import numpy as np
import torch, faiss

biencoder = SentenceTransformer('msmarco-distilbert-base-v4')
crossencoder = SentenceTransformer('cross-encoder/ms-marco-MiniLM-L-6-v2')

with torch.no_grad():
    embeddings = np.array([biencoder.encode(x) for x in sync_tqdm(metadata_neurips_abs.keys())])


No sentence-transformers model found with name cross-encoder/ms-marco-MiniLM-L-6-v2. Creating a new one with MEAN pooling.
100%|██████████| 3512/3512 [00:38<00:00, 90.85it/s]


## 得到和question相关的候选paper

In [286]:
def build_searcher(embeddings):
    index = faiss.IndexFlatL2(len(embeddings[0]))
    index.add(embeddings)
    def search(query_text, top_k):
        query_embedding = biencoder.encode(query_text)
        query_embedding = query_embedding / np.linalg.norm(query_embedding)
        distances, indices = index.search(query_embedding.reshape(1, -1), top_k)
        search.simlarities = 1 - (np.square(distances[0]) / 2)
        return indices[0]
    return search

searcher = build_searcher(embeddings)
rs = searcher(questions[0], 10)


In [64]:
from collections import Counter
rs = [(q,searcher(q, 10)) for q in sync_tqdm(questions)]
retrieve_freq = Counter(list(chain(*rs)))
retrieve_freq = {titles[x]:y for x,y in retrieve_freq.items()}
# retrieve_freq

In [ ]:
# TODO 相关性标签合成：用Claude评估question和初排结果的相关性；biencoder和crossencoder的微调

prompt_relv_gen = """

"""

In [ ]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from sentence_transformers.trainer import SentenceTransformerTrainer

from torch.utils.data import Dataset, DataLoader, random_split

